In [8]:
import requests
import json
import pymongo
from key import key
url = "https://api-basketball.p.rapidapi.com/"
headers = {
    "X-RapidAPI-Host": "api-basketball.p.rapidapi.com",
    "X-RapidAPI-Key": key
}

ModuleNotFoundError: No module named 'key'

In [9]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)


In [10]:
db = client.nba_db

In [11]:
seasons = ['2019-2020', '2020-2021', '2021-2022']
collection = db.standings
collection.delete_many({})
for season in seasons:
    params = f'standings?league=12&season={season}'
    response = requests.get(url + params, headers=headers).json()

    weststandings = {}
    eaststandings = {}

    for team in response["response"][0]:

        if team["group"]["name"] == "Western Conference":
            weststandings[team["team"]["name"]] = {"wins": team["games"]["win"]["total"],
                                  "losses": team["games"]["lose"]["total"], "logo": team["team"]["logo"]}
        if team["group"]["name"] == "Eastern Conference":
            eaststandings[team["team"]["name"]] = {"wins": team["games"]["win"]["total"],
                                  "losses": team["games"]["lose"]["total"], "logo" : team["team"]["logo"]}
    collection.insert_one({"season" : season, "data" : {"weststandings" : weststandings, "eaststandings": eaststandings}})



ServerSelectionTimeoutError: localhost:27017: [Errno 61] Connection refused, Timeout: 30s, Topology Description: <TopologyDescription id: 625195bbde1d470e425abe66, topology_type: Unknown, servers: [<ServerDescription ('localhost', 27017) server_type: Unknown, rtt: None, error=AutoReconnect('localhost:27017: [Errno 61] Connection refused')>]>

In [ ]:
idmap = {}
for team in response["response"][0]:
    idmap[team["team"]["id"]] = team["team"]["name"]

In [ ]:
idmap

In [ ]:
collection = db.h2h
for team1 in range(132, 162):
    for team2 in range(team1 + 1, 162):
        params = f"/games?{team1}-{team2}"
        response = requests.get(url + params, headers=headers).json()["response"]
        collection.insert_one({f'{idmap[team1]}-{idmap[team2]}' : response})
        

In [ ]:
with open('player_stats.json') as file:
    file_data = json.load(file)

In [ ]:
collection = db.player_data
collection.insert_many(file_data)